In [ ]:
# importamos librerias

# tratado de datos
import numpy as np
import pandas as pd

# visualizacion 
import matplotlib.pyplot as plt
import seaborn as sn

# deep learning
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

# funciones necesarias
import random
import os
import cv2
from skimage.io import imread

In [ ]:
#Cargamos nuestro data set
IMAGE_SIZE = 100
TRAIN_PATH = "path al data.train"
TEST_PATH = "path al data.test"

In [ ]:
# df_test 

def read_data(path):
    archivos = []
    for file in os.listdir(path): 
            archivos.append(file)            
    return np.array(archivos)
    
archivos_test = read_data(TEST_PATH)

df_test = pd.DataFrame({
    'archivo': archivos_test})


# df_train

df_train = pd.read_csv("path al data.target")

# One Hot Encoder 

from sklearn.preprocessing import LabelEncoder

df_train["target"] = df_train["target"].astype('category')
df_train["target_2"] = df_train["target"].cat.codes

In [ ]:
# Revisamos la si limpieza fue completada con exito
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   archivo   9000 non-null   object  
 1   target    9000 non-null   category
 2   target_2  9000 non-null   int8    
dtypes: category(1), int8(1), object(1)
memory usage: 89.5+ KB


In [ ]:
# Train
def read_data(df, path, image_size):
    X = []
    Y = []
    
    for index, row in df.iterrows():
        image = imread(os.path.join(path, row['archivo']))
        
        smallimage = cv2.resize(image, (IMAGE_SIZE,IMAGE_SIZE))
        X.append(smallimage)
        
        Y.append(row['target'])

    return np.array(X), np.array(Y)

X_train, y_train = read_data(df_train, TRAIN_PATH, IMAGE_SIZE)

In [ ]:
# Test
def read_data2(df, path, image_size):
    X = []
    Y = []
    
    for index, row in df.iterrows():
        image = imread(os.path.join(path, row['archivo']))
        
        smallimage = cv2.resize(image, (IMAGE_SIZE,IMAGE_SIZE))
        X.append(smallimage)

    return np.array(X)

X_test = read_data2(df_test, TEST_PATH, IMAGE_SIZE)

In [ ]:
# Escalamos nuestras imagenes
X_train_norma = X_train / 255.0
X_test_norma = X_test / 255.0

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# aumentamos nuestra cantidad de datos con fotos generadas a partir de las que ya tenemos
train_datagen = ImageDataGenerator(rescale = 1./255.,                                   
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# reescalamos el validation ya que no debe estar escalado
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [ ]:
# preparamos el entrenamiento
train_generator = train_datagen.flow_from_dataframe(df_train,
                                                    TRAIN_PATH,
                                                    x_col='archivo',
                                                    y_col='target',
                                                    batch_size = 20,
                                                    class_mode = 'sparse',
                                                    target_size = (IMAGE_SIZE, IMAGE_SIZE))

Found 9000 validated image filenames belonging to 29 classes.


"# Flow validation images in batches of 20 using test_datagen generator\nvalidation_generator = validation_datagen.flow_from_dataframe(validate_df,\n                                                              TRAIN_PATH,\n                                                              x_col='archivo',\n                                                              y_col='target_2',\n                                                              batch_size = 20,\n                                                              class_mode = 'sparse',\n                                                              target_size = (IM_SIZE, IM_SIZE))"

In [ ]:
# Creamos nuestra red neuronal para el modelo utilizando un modelo Pre-entrenado como el VGG16

from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
                  include_top=False)

for layer in base_model.layers:
    layer.trainable = False

    
# Capa de entrada
x = keras.layers.Flatten()(base_model.output)

# Capa de procesado
x = keras.layers.Dense(1000, activation='relu')(x)
x = keras.layers.Dense(1000, activation='sigmoid')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1000, activation='sigmoid')(x)
x = keras.layers.Dropout(0.5)(x)

# Capa de salida con 29 resultados ya que habian 29 letras en las imagenes anteriores
x = keras.layers.Dense(29, activation='softmax')(x)

# Terminamos el modelo usando el sparse_categorical_crossentropy
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

In [ ]:
# Corremos el modelo y vemos que tal la precision
vgghist = model.fit(train_generator,
                    epochs = 35,
                    batch_size=20)

In [ ]:
# Vemos las predicciones
predictions = model.predict(X_test_norma)
predict = predictions.argmax(axis=1)
predict

array([ 1, 21, 23, ..., 21,  3,  0], dtype=int64)

In [ ]:
df_test['target']=predict

In [ ]:
# Aqui podemos ver el resultado de las predicciones
series = {}

for i in range(0,29):
    A = df_train[df_train['target_2']==i]['target'].head(1)
    B =A.values
    
    series[B[0]]=i
    
print(series)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}
